In [74]:
### Requisitos para o funcionamento do Notebook:

# pip install selenium
# pip install bs4
# pip install requests
# pip install pandas
# pip install lxml
# pip install unidecode
# pip install tqdm

# Fazer o download do Webdriver em: https://chromedriver.storage.googleapis.com/index.html?path=102.0.5005.61/
# Colocar o arquivo chromedriver em um caminho de pastas sem espaços e preencher na variável driver_path

### Referencias para o Webdriver
# https://stackoverflow.com/questions/53973423/getting-value-after-button-click-with-beautifulsoup-python
# https://chromedriver.chromium.org/getting-started
# https://www.selenium.dev/documentation/webdriver/getting_started/first_script/
# https://selenium-python.readthedocs.io/waits.html
# https://www.tutorialspoint.com/get-html-source-of-webelement-in-selenium-webdriver-using-python

### Referencias para o BeautifulSoup
# https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start
# https://beautiful-soup-4.readthedocs.io/en/latest/#contents-and-children
# https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string
# https://beautiful-soup-4.readthedocs.io/en/latest/#installing-a-parser

#Referencias sobre o Pandas
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html?highlight=rename#pandas.DataFrame.rename
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html?highlight=join#pandas.DataFrame.join
# https://stackoverflow.com/questions/25952790/convert-pandas-series-from-dtype-object-to-float-and-errors-to-nans

### Referencias para o Regex
# https://www.w3schools.com/python/python_regex.asp

### Referencias para o tqdm
# https://tqdm.github.io/

### Referencias para o Unidecode
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait as wait # Não utilizado quando se usa o sleep.
import warnings
from time import sleep
from bs4 import BeautifulSoup
from requests import get
import re
import pandas as pd
import unidecode # para retirar os acentos
from tqdm import tqdm # para a barra de progresso
import sqlite3 as sql
from pathlib import Path

In [2]:
#Webdriver

def abrirWebdriver():
    warnings.filterwarnings('ignore')
    driver_path = 'C:/ProjetoAplicado/chromedriver'
    
    print("Abrindo o Webdriver...")
    driver = webdriver.Chrome(driver_path)
    
    return driver

def fecharWebdriver(driver):
    warnings.filterwarnings('ignore')
    
    print("Fechando o Webdriver...")
    driver.quit()

def obterSoupPorMunicipio(municipio, wait_in_seconds, driver):
    warnings.filterwarnings('ignore')

    #municipio = 'cabo-frio'
    #wait_in_seconds = 20
    url = 'https://cidades.ibge.gov.br/brasil/rj/'+municipio+'/panorama'
    
    print("Acessando a url...")
    driver.get(url)

    print("Aguardando",wait_in_seconds,"segundos para o carregamento da página...")
    #driver.implicitly_wait(10)
    #sleep(wait_in_seconds)
    
    for i in tqdm(range(wait_in_seconds)):
        sleep(1)

    print("Pegando o elemento principal pela classe...")
    elemento = driver.find_element(By.CLASS_NAME, "lista")
    #print(elemento.get_attribute('innerHTML'))

    print("Criando o Soup...")
    soup = BeautifulSoup(elemento.get_attribute('innerHTML'), 'html.parser')
    
    return soup

In [3]:
#BeautifulSoup

def obterDicionarioDeIndicadores(soup):

    print("Encontrando os elementos na página")
    indicadores = soup.find_all("tr", class_=re.compile("lista__indi"))

    ind = []
    values = []
    flag_ind = True

    print("Varrendo a árvore de elementos")
    for indicador in indicadores:
        for child in indicador.contents:
            if str(type(child)) == "<class 'bs4.element.Tag'>" and child.text != "":
                text = " ".join(child.text.strip().replace("\n"," ").split())
                #print(text)
                if flag_ind:
                    ind.append(text)
                    flag_ind = False
                    #print("Indicador encontrado:",text)
                else:
                    text = re.sub(',','.',re.sub('\.','',re.split("\s", text, 1)[0]))
                    values.append(text)
                    flag_ind = True
    print("Foram encontrados",len(ind),"indicadores")

    print("Criando o Dicionário com os indicadores")
    dic = {
        "indicador" : ind,
        "Valor" : values
    }
    
    return dic

In [5]:
def obterParametroMunicipios():
    print("Carregando Municipios do IBGE...")
    
    url = 'https://ibge.gov.br/explica/codigos-dos-municipios.php'

    municipios = pd.read_html(url, match='Municípios do Rio de Janeiro')[0]
    
    municipios[['parametro']] = municipios[['Municípios do Rio de Janeiro']]

    municipios.parametro = municipios.parametro.str.lower().str.replace(' ','-')
    
    return municipios.parametro

In [3]:
# Criação dos bancos de dados

# Utiliza o mesmo endereço onde está o rodando o script
Endereco = Path.cwd()

BD = Endereco / 'projeto.db'

if Endereco.exists():
    if (BD.exists()):
        print ('Banco de dados já existe')
    else:
        BD.touch()
        print ('Banco de dados criado com sucesso')
else:
    print ('Endereço não existe')

Banco de dados já existe


In [12]:
driver = abrirWebdriver()

municipio = obterParametroMunicipios()[0]

print(">>>>>>>>>>", unidecode.unidecode(municipio))

soup = obterSoupPorMunicipio(unidecode.unidecode(municipio),20,driver)

dic = obterDicionarioDeIndicadores(soup)

dfIndicadores = pd.DataFrame(dic)

dfIndicadores['Valor'] = pd.to_numeric(dfIndicadores['Valor'], errors='coerce').astype('float64')
    
dfIndicadores = dfIndicadores.rename(columns={"Valor" : unidecode.unidecode(municipio)})


Abrindo o Webdriver...
Carregando Municipios do IBGE...
>>>>>>>>>> angra-dos-reis
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]

Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores


In [13]:
for municipio in obterParametroMunicipios()[1:]:
    print("\n>>>>>>>>>>", unidecode.unidecode(municipio))
    soup = obterSoupPorMunicipio(unidecode.unidecode(municipio),20,driver)

    dic = obterDicionarioDeIndicadores(soup)

    other = pd.DataFrame(dic)
    
    dfIndicadores = dfIndicadores.join(other[["Valor"]])
    
    dfIndicadores['Valor'] = pd.to_numeric(dfIndicadores['Valor'], errors='coerce').astype('float64')
    
    dfIndicadores = dfIndicadores.rename(columns={"Valor" : unidecode.unidecode(municipio)})

fecharWebdriver(driver)    
    
print("\nWebscrapper finalizado com sucesso!")

Carregando Municipios do IBGE...

>>>>>>>>>> aperibe
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores

>>>>>>>>>> araruama
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores

>>>>>>>>>> areal
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


...

Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores

>>>>>>>>>> vassouras
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores

>>>>>>>>>> volta-redonda
Acessando a url...
Aguardando 20 segundos para o carregamento da página...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Pegando o elemento principal pela classe...
Criando o Soup...
Encontrando os elementos na página
Varrendo a árvore de elementos
Foram encontrados 37 indicadores
Criando o Dicionário com os indicadores
Fechando o Webdriver...

Webscrapper finalizado com sucesso!


In [14]:
#renomear o índice
dfIndicadores.index.name = 'id'

#índice começando em 1
dfIndicadores.index = dfIndicadores.index + 1

dfIndicadores = dfIndicadores[:29]

print("Conectando no banco de dados...")
ConexaoBD = sql.connect(BD)

print("filtrando as linhas de interesse...")
dfIndicadores = dfIndicadores[:29]

#dfIndicadores.to_sql('indicadores_ibge_municipios', ConexaoBD, if_exists='replace')
#print("Foram carregadas",dfIndicadores.shape[0], "linhas em", dfIndicadores.shape[1], "colunas no banco." )

print("Encerrando a conexão com o banco...")
ConexaoBD.close()

Conectando no banco de dados...
filtrando as linhas de interesse...
Foram carregadas 29 linhas em 93 colunas no banco.
Encerrando a conexão com o banco...


In [15]:
dfIndicadores

,indicador,angra-dos-reis,aperibe,araruama,areal,armacao-dos-buzios,arraial-do-cabo,barra-mansa,barra-do-pirai,belford-roxo,...,silva-jardim,sumidouro,tangua,teresopolis,trajano-de-moraes,tres-rios,valenca,varre-sai,vassouras,volta-redonda
id,,,,,,,,,,,,,,,,,,,,,
1,População estimada [2021],210171.000,12036.000,136109.000,12763.000,35060.000,30827.000,185237.000,101139.000,515239.000,...,21775.000,15709.000,34898.000,185820.000,10653.000,82468.000,77202.000,11208.000,37262.000,274925.000
2,População no último censo [2010],169511.000,10213.000,112008.000,11423.000,27560.000,27715.000,177813.000,94778.000,469332.000,...,21349.000,14900.000,30732.000,163746.000,10289.000,77432.000,71843.000,9475.000,34410.000,257803.000
3,Densidade demográfica [2010],205.450,107.920,175.550,102.990,392.160,172.910,324.940,163.700,6031.380,...,22.770,37.670,211.210,212.490,17.440,237.420,55.060,49.850,63.940,1412.750
4,Salário médio mensal dos trabalhadores formais...,3.000,1.600,1.800,2.200,1.900,1.900,2.000,1.700,2.300,...,2.100,2.200,2.000,2.000,2.000,1.800,2.000,2.100,2.200,2.100
5,Pessoal ocupado [2020],35280.000,2067.000,23123.000,3440.000,14209.000,6578.000,35733.000,17576.000,31831.000,...,4189.000,2002.000,4441.000,41414.000,1542.000,24118.000,12296.000,1210.000,8601.000,80394.000
6,População ocupada [2020],17.000,17.400,17.200,27.200,41.200,21.500,19.300,17.400,6.200,...,19.200,12.800,12.800,22.500,14.500,29.400,16.000,10.900,23.200,29.300
7,Percentual da população com rendimento nominal...,34.900,33.100,36.100,35.900,31.100,35.400,34.500,33.800,39.300,...,39.600,37.000,38.300,32.300,35.500,31.600,32.600,34.900,34.200,32.700
8,Taxa de escolarização de 6 a 14 anos de idade ...,96.400,96.700,98.300,98.000,97.600,99.200,98.400,98.700,96.200,...,98.800,93.500,98.100,96.200,98.400,97.900,97.700,96.800,98.800,98.500
9,IDEB – Anos iniciais do ensino fundamental (Re...,5.100,5.500,5.400,6.000,5.800,5.600,5.200,5.400,4.700,...,5.200,4.900,4.900,6.000,5.800,6.000,5.000,5.300,5.200,6.200
